In [1]:
import numpy as np
import struct 

#This reads the MNIST dataset files in IDX1 format.
def read_idx1_file(filename):
    with open(filename, 'rb') as f:
        magic, num_of_labels = struct.unpack('>II', f.read(8))
        assert magic == 2049, f"Expected magic number 2049, got {magic}"

        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

#This reads the MNIST dataset files in IDX3 format.
def read_idx3_file(filename):
    with open(filename, 'rb') as f:
        magic, num_of_images, rows, cols = struct.unpack('>IIII', f.read(16))
        assert magic == 2051, f"Expected magic number 2051, got {magic}"

        images = np.frombuffer(f.read(), dtype=np.uint8)
        images = images.reshape(num_of_images, rows, cols)
    return images


In [2]:
## Read the MNIST dataset
train_labels = read_idx1_file('handwritten_numbers_dataset/train-labels-idx1-ubyte/train-labels-idx1-ubyte')
train_images = read_idx3_file('handwritten_numbers_dataset/train-images-idx3-ubyte/train-images-idx3-ubyte')

test_labels = read_idx1_file('handwritten_numbers_dataset/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')
test_images = read_idx3_file('handwritten_numbers_dataset/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte')

In [3]:
# Normalize the images to the range [0, 1] for better training performance
train_images = train_images/255.0
test_images = test_images/255.0

In [4]:
# Reshape the images to be 2D arrays
train_images = train_images.reshape(train_images.shape[0], -1)
test_images = test_images.reshape(test_images.shape[0], -1)

In [5]:
from sklearn.model_selection import train_test_split
train_images, Val_images, train_labels, Val_labels = train_test_split(train_images, train_labels, test_size=0.1667, random_state=42)

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score

#hyperparameter tuning for SVM and Logistic Regression

# Parameter grid for SVM
param_grid_svm = {
    'C': [0.1, 10],
    'kernel': ['linear'],
    'gamma': ['scale']
}

# Create a SVM classifier
svc = SVC()
grid_svc = GridSearchCV(svc, param_grid_svm, cv=5)
grid_svc.fit(train_images, train_labels)

#Best SVM model
best_svc = grid_svc.best_estimator_

# Evaluate the best SVM model on the validation set
val_predictions_svm = best_svc.predict(Val_images)
val_accuracy_svm = accuracy_score(Val_labels, val_predictions_svm)
val_f1_svm = f1_score(Val_labels, val_predictions_svm, average='weighted')

print("SVM - Validation Accuracy:", val_accuracy_svm)
print("SVM - Validation F1-Score:", val_f1_svm)

# Evaluate the best SVM model on the test set
test_predictions_svm = best_svc.predict(test_images)
test_accuracy_svm = accuracy_score(test_labels, test_predictions_svm)
test_f1_svm = f1_score(test_labels, test_predictions_svm, average='weighted')

print("SVM - Test Accuracy:", test_accuracy_svm)
print("SVM - Test F1-Score:", test_f1_svm)



SVM - Validation Accuracy: 0.943011397720456
SVM - Validation F1-Score: 0.942856096943864
SVM - Test Accuracy: 0.9447
SVM - Test F1-Score: 0.9445740358444961


In [7]:
#Saving my model 

import pickle
import logging  # It's good to log these operations

def save_model(model, filename):
    """Saves the model to a pickle file."""
    try:
        with open(filename, 'wb') as f:  # 'wb' mode for writing binary files
            pickle.dump(model, f)
        logging.info(f"Model saved to {filename}")
    except Exception as e:
        logging.error(f"Error saving model: {e}", exc_info=True)  # Log the full error

# Example usage after training your model:
save_model(best_svc, "my_trained_svm_model.pkl")
